<a id="1"></a> <br>
# 1. Introduction

![](http://www.mydr.com.au/wp-content/uploads/2019/04/heart_failure_750.jpeg)

![](http://)

Heart failure is a pathophysiological state in which cardiac output is insufficient to meet the needs of the body and lungs. The term "congestive heart failure" is often used, as one of the common symptoms is congestion, or build-up of fluid in a person's tissues and veins in the lungs or other parts of the body

in this study we want to analyse this small data and detect relation between each feature and target 
then we visulate the important information of data 
finnaly we build model for our data try to get better modelisation for our dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="2"></a> <br>
# 2. reading data and exploration 

In [ ]:
data = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
print("dimonsion of the data set is ",data.shape) 
data.info()
data.head()

In [ ]:
data.dtypes.value_counts()

In [ ]:
data.describe()

In [ ]:
print("data set already cleaned we don't have any missing values")
data.isnull().sum()

<a id="3"></a> <br>
# 3. Exploratory data analysis

In [ ]:
data.columns

In [ ]:
feature= ['anaemia','diabetes','high_blood_pressure','sex','smoking']
for i in feature :
    print(data[i].value_counts())
    plt.figure(figsize=(6,6))
    data[i].value_counts().plot(kind='pie',legend=True,autopct='%1.0f%%',colors=["g","y"])

In [ ]:
print('percentage of the most ages present in this stady')
plt.figure(figsize=(16, 9))
data["age"].value_counts().head(5).plot(kind="bar",title='percentage of the most ages present in this stady',color='r')
(data["age"].value_counts()*100/len(data)).head(5)

In [ ]:
plt.figure(figsize=(16, 9))
data["age"].describe()[1:].plot(kind="bar",grid=True)

In [ ]:
data[['age','DEATH_EVENT']].groupby("age").sum().plot(grid=True,figsize=(16, 9),title="number of death_events per age",color="b")

as we can see in this figure the highest number of death_event is on age of 60 and thier nearly 
many people in this age retired i think this death_event is resultat of pression of thier carreer
but honstly i don't see relation between the deeth_event and age because in age of 47 we have 7 events and the same in the age of 80

In [ ]:
for col in data.select_dtypes("float"):
    plt.figure()
    sns.distplot(data[col])

**comment and mark for distrubtions**
* data normalised for the three feature (age , platelates ,sercum_creatinine)

In [ ]:
feauture = ['anaemia', 'diabetes', 'high_blood_pressure','sex', 'smoking']
for i in feauture :
    plt.figure(figsize=(4,4))
    sns.heatmap(pd.crosstab(data["DEATH_EVENT"],data[i]),annot=True)

**note and comment**
* we have 83 person have an anemie but he don' died and we have 46 person have an anemie and died but with this results we can't link the death_event to the anemia perhaps it is an  fearture combined with some causes but its self alone can't provid heart faillure
* the same way with the other feature (sex,blood_pression,diabtes....)  sometimes we detect a death_event in their presence but some times the opposites so in conclusion we can say that this variation of value of this features can detect a death_event but heart_faillure not linked directly to one feature(cause) 

In [ ]:
data.corr().abs()

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(data.corr().abs())

**mark and note**
* bad correlation  beetween feature expect between  time and some other feature
* so for this one reason in the next modelistaion we use all feature 
* in part of feature selection i use all of feature because we detect generally  the same correlation between each feature and the target 

<a id="2"></a> <br>
# 2. machine learning part (building,training _testing  the model )

# fist one we build some naive models and we try to imporove it by changing thier parametrs

# *1/logitic regression model*

In [ ]:
data.columns

In [ ]:
feature = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']
target =['DEATH_EVENT']
X = data[feature].values
y = data[target].values.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=42)
model = LogisticRegression()
model.fit(X_train,y_train)
ypredict = model.predict(X_test)
cm = metrics.confusion_matrix(y_test, ypredict)
print("Accuracy:",metrics.classification_report(y_test, ypredict))
sns.heatmap(cm,annot=True)

# 2 decision treeregression

In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train,y_train)
ypredict = model.predict(X_test)
cm = metrics.confusion_matrix(y_test, ypredict)
print("Accuracy:",metrics.classification_report(y_test, ypredict))
sns.heatmap(cm,annot=True)

# knn models

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)
ypredict = model.predict(X_test)
cm = metrics.confusion_matrix(y_test, ypredict)
print("Accuracy:",metrics.classification_report(y_test, ypredict))
sns.heatmap(cm,annot=True)

# random forest models

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
ypredict = model.predict(X_test)
cm = metrics.confusion_matrix(y_test, ypredict)
print("Accuracy:",metrics.classification_report(y_test, ypredict))
sns.heatmap(cm,annot=True)

**model selection **

1- decisiontreeclassifer

In [ ]:
param_grid = { 'criterion': ["gini", "entropy"] ,'max_depth':[5, 10]}
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(X_train, y_train) 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(metrics.classification_report(y_test, grid_predictions)) 

2/knn model

In [ ]:
param_grid = {"weights":["uniform", "distance"], 
              "n_neighbors":[2,3,5,7,9,11,15]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(X_train, y_train) 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(metrics.classification_report(y_test, grid_predictions)) 

3/random_foreset_models

In [ ]:
param_grid = {'n_estimators': [100, 200, 500, 1000],  
              'criterion': ["gini", "entropy"], 
              'max_depth':[5, 7]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(X_train, y_train) 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(metrics.classification_report(y_test, grid_predictions)) 

**summury of resulats of part of builidin models**

In [ ]:
accurency_before = [0.8,0.63,0.73,0.53]
acuurency_after = [0.8,0.72,0.75,0.53]
model_name = ["logistic_regression","tree_classifer",'randomforestclassifer', 'knn_classifer' ]
e = np.arange(4)
width = 0.2
  
# plot data in grouped manner of bar type
plt.figure(figsize=(16,10))
plt.bar(e-0.2, accurency_before, width, color='b')
plt.bar(e, acuurency_after, width, color='y')
plt.xticks(e, model_name)
plt.xlabel("models")
plt.ylabel("accurency")
plt.legend(["before", "after"])
plt.grid()
plt.show()

<a id="5"></a> <br>
# 5. conclusion

as we can see in these figure and based on these analysis the best accurency can get with this dataset is 80% 
i think that some people talk about feature selection here and use it and get accurency better to 80% but in my opinion you can't build your model and chose some fearture and leave the rest  beacause as we see in part of analysis correlation between feature and target are genarally the same , then these kinds  of these data need an expert from the field to say if we cant expect variable from the buildin model operation or not 
as resume i say you can't build mathematical model with an accurancy grandther to 80%  but you can't applied ti the real world 


if you found my work awsome give an upvote please 